In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import statistics as stats
import scipy
from scipy.stats.stats import pearsonr
from matplotlib import style
import warnings
import seaborn as sns
import requests
import bs4 as bs
import os
import csv
from openpyxl.workbook import Workbook as wb
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
warnings.filterwarnings('ignore') 

In [2]:
'''USE FF_Research_Data, col (RM-RF) + col (RF) = RM(t) to generate beta
Annual Excess Return: rp(t) – RF(t) , where RF(t) comes from FF.
R_j(t) = RF(t) + beta_j*( RM-RF  ), rewrite differently
R_j(t) – RF(t) =  ( RM(t)-RF(t)  )*beta_j'''

'USE FF_Research_Data, col (RM-RF) + col (RF) = RM(t) to generate beta\nAnnual Excess Return: rp(t) – RF(t) , where RF(t) comes from FF.\nR_j(t) = RF(t) + beta_j*( RM-RF  ), rewrite differently\nR_j(t) – RF(t) =  ( RM(t)-RF(t)  )*beta_j'

In [3]:
data = pd.read_excel('stock_returns_data.xlsx')

In [4]:
df = data[['Names Date','Ticker Symbol','Returns']]

In [5]:
df = df[pd.to_numeric(df['Returns'], errors='coerce').notnull()]

In [6]:
df['Date'] = df['Names Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
#df['Date'] = df['Names Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
df
#date = dt(year=int(dates[0:4]), month=int(dates[4:6]), day=int(dates[6:8]))
#int2date(dates)

,Names Date,Ticker Symbol,Returns,Date
0,20070131,EWST,0.023279,2007-01-31
1,20070228,EWST,0.261621,2007-02-28
2,20070330,EWST,0.019732,2007-03-30
3,20070430,EWST,0.002764,2007-04-30
4,20070531,EWST,0.043418,2007-05-31
...,...,...,...,...
1048568,20191129,VT,0.025614,2019-11-29
1048569,20191231,VT,0.034494,2019-12-31
1048572,20080731,SZR,0.076445,2008-07-31
1048573,20080829,SZR,-0.040735,2008-08-29


In [7]:
df['Returns']=df['Returns']*100
df1 = df[['Date','Ticker Symbol','Returns']]
df1['Date'] = df1['Date'].map(lambda x: 100*x.year + x.month)
df1['Date'] = df1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
df1

,Date,Ticker Symbol,Returns
0,2007-01-01,EWST,2.3279
1,2007-02-01,EWST,26.1621
2,2007-03-01,EWST,1.9732
3,2007-04-01,EWST,0.2764
4,2007-05-01,EWST,4.3418
...,...,...,...
1048568,2019-11-01,VT,2.5614
1048569,2019-12-01,VT,3.4494
1048572,2008-07-01,SZR,7.6445
1048573,2008-08-01,SZR,-4.0735


In [8]:
#df1 = df1[(df1['Date'] > '2015-01-31') & (df1['Date'] < '2020-01-31')]
df1 = df1[(df1['Date'] > '2014-12') & (df1['Date'] < '2020-01')]
df1.head(50)

,Date,Ticker Symbol,Returns
96,2015-01-01,EGAS,-11.5245
97,2015-02-01,EGAS,3.5897
98,2015-03-01,EGAS,-0.0495
99,2015-04-01,EGAS,1.1044
100,2015-05-01,EGAS,-0.6951
101,2015-06-01,EGAS,3
102,2015-07-01,EGAS,-1.3107
103,2015-08-01,EGAS,-11.1665
104,2015-09-01,EGAS,0.3367
105,2015-10-01,EGAS,0.1678


In [9]:
df1 = df1.dropna(subset=['Ticker Symbol'])

In [10]:
fama = pd.read_excel('FF_Research_Data_Factors.xlsx')
fama = fama[['Date','Mkt-RF','RF']]
fama['Returns'] = fama['Mkt-RF']+fama['RF']
fama['Ticker Symbol']='SP500'
fama1 = fama[['Date','Ticker Symbol','Returns','RF']]
fama1['Date'] = fama1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m'))
fama1

,Date,Ticker Symbol,Returns,RF
0,1926-07-01,SP500,3.18,0.22
1,1926-08-01,SP500,2.89,0.25
2,1926-09-01,SP500,0.59,0.23
3,1926-10-01,SP500,-2.92,0.32
4,1926-11-01,SP500,2.84,0.31
...,...,...,...,...
1120,2019-11-01,SP500,3.99,0.12
1121,2019-12-01,SP500,2.91,0.14
1122,2020-01-01,SP500,0.02,0.13
1123,2020-02-01,SP500,-8.01,0.12


In [11]:
dfAll = pd.concat([df1,fama1])
dfAll = dfAll.reset_index()
#dfAll = dfAll[['Date','Ticker Symbol', 'Returns']]
dfAll = dfAll[(dfAll['Date'] > '2014-12-31') & (dfAll['Date'] < '2019-12-31')]
dfAll = dfAll[['Date','Ticker Symbol','Returns','RF']]
#dfAll = dfAll.set_index('Date')
dfAll

,Date,Ticker Symbol,Returns,RF
0,2015-01-01,EGAS,-11.5245,NaN
1,2015-02-01,EGAS,3.5897,NaN
2,2015-03-01,EGAS,-0.0495,NaN
3,2015-04-01,EGAS,1.1044,NaN
4,2015-05-01,EGAS,-0.6951,NaN
...,...,...,...,...
413746,2019-08-01,SP500,-2.42,0.16
413747,2019-09-01,SP500,1.62,0.18
413748,2019-10-01,SP500,2.21,0.15
413749,2019-11-01,SP500,3.99,0.12


In [17]:
#finding stocks in the data (excel sheet) that have data from 2015-2019 (returns) and 2015-2017 (beta)
stock_count = dfAll.groupby('Ticker Symbol')['Returns'].count()
stock_count = stock_count[stock_count == 60]
stocks = [*stock_count.keys()]
stocks

['A',
 'AADR',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABDC',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABT',
 'ACAD',
 'ACC',
 'ACCO',
 'ACGL',
 'ACH',
 'ACHC',
 'ACHN',
 'ACIW',
 'ACLS',
 'ACM',
 'ACN',
 'ACNB',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRX',
 'ACST',
 'ACTG',
 'ACU',
 'ACWI',
 'ACWV',
 'ACWX',
 'ACY',
 'ADBE',
 'ADC',
 'ADI',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRU',
 'ADS',
 'ADSK',
 'ADTN',
 'ADX',
 'ADXS',
 'AE',
 'AEE',
 'AEG',
 'AEGN',
 'AEHR',
 'AEIS',
 'AEL',
 'AEM',
 'AEO',
 'AEP',
 'AER',
 'AERI',
 'AES',
 'AEY',
 'AEZS',
 'AFB',
 'AFG',
 'AFH',
 'AFL',
 'AFMD',
 'AFT',
 'AG',
 'AGCO',
 'AGD',
 'AGEN',
 'AGG',
 'AGI',
 'AGIO',
 'AGNC',
 'AGND',
 'AGO',
 'AGRO',
 'AGRX',
 'AGTC',
 'AGX',
 'AGYS',
 'AGZD',
 'AHC',
 'AHH',
 'AHPI',
 'AHT',
 'AI',
 'AIA',
 'AIF',
 'AIG',
 'AIMC',
 'AIN',
 'AINC',
 'AINV',
 'AIR',
 'AIRI',
 'AIRR',


In [85]:
#Dataframe with all the stocks that have all the required data (date[2015,2019],monthly returns)
dfAll1 = dfAll.loc[dfAll['Ticker Symbol'].isin(stocks)]
dfAll1.tail(60)

,Date,Ticker Symbol,Returns,RF
413691,2015-01-01,SP500,-3.11,0.00
413692,2015-02-01,SP500,6.13,0.00
413693,2015-03-01,SP500,-1.12,0.00
413694,2015-04-01,SP500,0.59,0.00
413695,2015-05-01,SP500,1.36,0.00
413696,2015-06-01,SP500,-1.53,0.00
413697,2015-07-01,SP500,1.54,0.00
413698,2015-08-01,SP500,-6.04,0.00
413699,2015-09-01,SP500,-3.08,0.00
413700,2015-10-01,SP500,7.75,0.00


In [19]:
###Beta calculations###
###Want monthly return beta from 2015 to 2017###
dfAll1.reset_index()
dfBeta = dfAll1[(dfAll1['Date'] > '2014-12-31') & (dfAll1['Date'] < '2017-12-31')]
dfBeta = dfBeta.reset_index()
dfBeta = dfBeta[['Date','Ticker Symbol','Returns','RF']]
dfBeta

,Date,Ticker Symbol,Returns,RF
0,2015-01-01,JJSF,-9.7913,NaN
1,2015-02-01,JJSF,3.1288,NaN
2,2015-03-01,JJSF,5.801,NaN
3,2015-04-01,JJSF,-2.2212,NaN
4,2015-05-01,JJSF,3.326,NaN
...,...,...,...,...
159403,2017-08-01,SP500,0.25,0.09
159404,2017-09-01,SP500,2.6,0.09
159405,2017-10-01,SP500,2.34,0.09
159406,2017-11-01,SP500,3.2,0.08


In [20]:
stdev_stock = dfBeta.groupby('Ticker Symbol')['Returns'].std()
stdev_stock

Ticker Symbol
A        6.151584
AADR     4.083586
AAL      9.241575
AAMC    30.317984
AAME     8.044327
          ...    
ZNGA     9.399612
ZNH     15.458949
ZTR      3.265853
ZTS      5.680491
ZUMZ    14.955430
Name: Returns, Length: 4428, dtype: float64

In [21]:
stdev_market = stdev_stock['SP500']
stdev_market

2.9964700926044654

In [22]:
market_quotient = (stdev_stock / stdev_market)
market_quotient['AAPL']

2.2222782891012445

In [23]:
###table to find correlations
symbols = dfBeta.groupby('Ticker Symbol')
market_df = symbols.get_group('SP500')
market_df = market_df[['Date','Returns']].set_index('Date')
table = dfBeta.pivot(index='Date',columns='Ticker Symbol',values='Returns')  
table
#table.to_excel('betaReturnsNYSE.xlsx')

Ticker Symbol,A,AADR,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,...,ZION,ZIOP,ZIXI,ZMLP,ZN,ZNGA,ZNH,ZTR,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,-7.4988,-2.5773,-8.4841,-48.4071,-0.9926,3.5656,-20.3209,-2.5904,-0.1758,6.1424,...,-15.9593,76.5286,-1.6667,0.4197,9.4891,-3.7594,10.8787,-1.2134,-0.5043,-3.4688
2015-02-01,11.7554,7.5791,-2.2005,10.6563,0.2506,-5.8433,39.5973,3.2095,-2.5598,10.0461,...,11.7488,25.8101,12.9944,2.706,30.6667,-10.1563,-5.9245,2.3127,7.8633,4.103
2015-03-01,-1.3267,-1.2917,10.1879,4.5637,0.75,-4.9547,11.218,8.9738,-3.3434,-3.1372,...,0.9912,-4.3517,-1.75,-4.6253,-11.2245,23.913,44.5648,-1.495,0.4339,3.6837
2015-04-01,-0.4332,4.0653,-8.327,20.9961,-1.2407,20.0989,2.8818,-2.2829,-4.4692,0.5786,...,4.9444,-18.1987,8.397,6.4739,16.092,-14.0351,36.071,-0.0804,-3.8605,-21.2174
2015-05-01,-0.4351,0.0124,-12.2502,-23.933,-4.0404,3.0294,25.07,-1.2516,7.1469,4.5146,...,2.1352,7.605,7.7465,-4.9292,-1.9802,20.4082,1.407,-0.4492,12.0441,-5.8341
2015-06-01,-6.0937,-3.6773,-5.747,-15.3178,-4.7368,3.4342,-2.7996,-4.3937,4.0008,-3.7266,...,9.8857,26.5823,12.6362,-10.1211,-3.5354,-3.0509,17.3135,-2.9859,-3.1143,-10.8172
2015-07-01,6.1431,3.6491,0.4132,-8.1225,2.7624,2.1265,12.0392,-1.4654,9.3666,-3.2888,...,-1.7173,11.3333,-5.6093,-5.0702,-7.8534,-13.2867,-14.9349,-0.9505,1.5761,-1.9902
2015-08-01,-11.3309,-6.8414,-2.5436,-70.9663,-3.2258,1.8118,5.964,-6.8049,0.597,-6.6117,...,-6.8291,-34.8054,-10.2459,-4.5993,-15.3409,2.0161,-36.6937,-5.7659,-8.2217,-10.6897
2015-09-01,-5.1776,-4.1935,-0.3848,-37.776,12.2222,-4.0292,-8.8792,-6.2863,8.1826,-2.1816,...,-5.0345,3.4443,-3.8813,-13.5602,-4.0268,-9.8814,17.142,-3.4341,-8.2238,-32.9472


In [80]:
table['RiskFree'] = fama1['RF'].iloc[-63:-27].values
dfObj = table.columns.values
fama1['mktExcessReturn'] = fama1['Returns'] - fama1['RF']
df_beta=[]
for obj in dfObj:
    dfExcessReturn = table[obj]-table['RiskFree']
    dfExcessReturn = dfExcessReturn.to_frame(name=obj)
    beta = dfExcessReturn[obj].sum()/fama1['mktExcessReturn'].iloc[-63:-27].sum()
    df_beta.append(
        {
            'Ticker': obj,
            'Beta': beta,
            'Excess Return': dfExcessReturn[obj].sum()
        }
    )

df_beta = pd.DataFrame(df_beta)
df_beta.to_excel('beta_df.xlsx')

In [89]:
###Find the returns for each stock that had data from 2015-2019
###returns will be calculated from 1-1-2018 - 11-29-2019 [OR 1.91667 years]
df_stock_returns = dfAll1[(dfAll1['Date'] > '2017-12-20') & (dfAll1['Date'] < '2019-12-31')]
stock_annual_returns = df_stock_returns.loc[df_stock_returns['Ticker Symbol'].isin(stocks)]
#stock_annual_returns['ExcessReturn']=stock_annual_returns['Returns']-
stock_annual_returns.pivot(index='Date',columns='Ticker Symbol',values='Returns')

Ticker Symbol,A,AADR,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,...,ZION,ZIOP,ZIXI,ZMLP,ZN,ZNGA,ZNH,ZTR,ZTS,ZUMZ
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,9.6461,6.0663,4.4013,-14.7059,6.6176,2.6098,-14.3575,-0.8174,17.3538,-1.0636,...,6.2955,-4.8309,-3.4247,7.7239,-2.3148,-10.5,24.8553,-2.6642,6.6852,-0.3601
2018-02-01,-6.5913,-3.14,0.0552,-6.6092,-4.8276,13.0105,-13.7697,0.9615,-2.3421,6.761,...,2.1099,-5.33,-4.2553,-10.5636,92.4171,-3.352,2.1793,-10.1083,5.3825,-5.0602
2018-03-01,-2.4639,-2.8127,-4.2212,-4.1923,-4.3478,0.9089,-10.2757,6.1224,3.8162,-5.8051,...,-4.075,5.0938,5.4321,-6.8132,17.2414,5.7803,-20.3903,-3.5361,3.2773,21.3198
2018-04-01,-1.5112,-0.5362,-17.3788,5.9815,-0.9091,-10.3648,27.5339,-12.8205,-3.4585,-1.502,...,3.8308,7.6531,18.267,6.1748,-9.2437,-5.7377,2.7171,-0.2457,0.115,-2.0921
2018-05-01,-5.8108,-1.3178,1.6539,4.3939,-9.2308,-4.7642,46.3392,-10.2941,12.381,13.5181,...,0.5479,16.1138,7.1287,3.2917,-17.5926,27.5362,-8.842,4.0369,0.2635,3.6325
2018-06-01,-0.1292,-7.1733,-12.8158,1.2337,-8.4746,9.3012,-3.9983,9.541,5.5512,-0.9418,...,-3.8679,-38.3673,-0.3697,-0.3259,13.9045,-7.5,-18.1983,1.8613,1.7802,3.299
2018-07-01,7.0327,-0.7325,4.1623,-6.0215,-3.7037,-0.3222,-14.4098,13.5338,4.0752,2.7983,...,-1.8789,-14.2384,-0.9276,5.747,-23.7978,-6.8796,-10.2454,5.2618,1.6622,-9.5808
2018-08-01,2.2714,-0.9271,2.6302,-5.3394,5.7692,14.8003,7.6243,7.0199,16.1439,20.0063,...,3.6557,12.7413,3.9326,1.0621,-40.1294,9.7625,-6.1443,2.8255,4.7641,37.5276
2018-09-01,4.4418,2.6943,2.0998,-3.5455,-9.0909,9.5937,-40.3772,-6.4356,2.6581,-0.8303,...,-5.8923,9.589,0,-0.6354,-30.8108,-3.6058,-3.7838,-5.5644,1.0596,-15.4093


In [91]:
table1 = stock_annual_returns.pivot(index='Date',columns='Ticker Symbol',values='Returns') 
#table1.dtypes
table1 = table1.astype(float)
total_ret = table1.sum()
annual_returns_stocks = total_ret/2
annual_returns_stocks.to_excel('stock_returns_NYSE.xlsx')

In [93]:
###sort betas into 34 portfolio's with equal weights and find total return
#34 because it is divisible by 4454 (number of tickers)
dataframe = pd.read_excel('beta_df_new.xlsx')

In [98]:
###beta of each 34 equally weighted portfolios of 131 stocks
group_size = 442
#group_size = 131
portfolios = [dataframe.iloc[i:i+group_size] for i in range(0,len(dataframe)-group_size+1,group_size)]   
for portfolio in portfolios:
    print(portfolio['Beta'].mean())

4.0262201568286216
2.1963523740501305
1.656353532550492
1.2547289782700355
0.9771374196963921
0.7731041006871533
0.5326982753162638
0.2334361726919912
-0.2794333384073855
-2.246961186196604


In [99]:
###annual returns of each portfolio sorted from highest to lowest beta
group_size = 442
portfolios = [dataframe.iloc[i:i+group_size] for i in range(0,len(dataframe)-group_size+1,group_size)]   
for portfolio in portfolios:
    print(portfolio['Returns'].mean())

5.174529298642535
10.463014705882351
9.616864932126703
8.854974321266962
8.244737895927601
6.85005667420815
5.052601809954752
4.345012217194567
1.6256941043083883
-0.14269762443439113


In [ ]:
###https://github.com/RJT1990/Active-Portfolio-Management-Notes/blob/master/Chapter%202%2C%20CAPM.ipynb 
#this link is somewhat what professor Luo was talking about